In [51]:
# Подключение пакетов
import os, sys, time
import uproot3 as uproot
import numpy as np
import pandas as pd
from math import floor
from mpl_toolkits import mplot3d
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from numbers import Integral
from scipy.optimize import curve_fit
from scipy.stats import norm, truncnorm, foldnorm
from itertools import compress
from pynverse import inversefunc
import warnings
from time import perf_counter


warnings.filterwarnings("ignore")

plt.rcParams['font.size'] = 16

print('Uproot version:',uproot.version.version)
print('Numpy version:', np.version.version)
print('Pandas version:', pd.__version__)
plt.ioff()

# инициализация генератора псевдослучайных чисел
# pandas version was 1.5.1

rng = np.random.default_rng(12345)

Uproot version: 3.14.4
Numpy version: 1.23.4
Pandas version: 2.0.0


In [52]:
datadir = 'data'
picsdir = 'pics'

In [53]:
# отображение ключей в файле uproot в виде иерархии
def show_uproot_tree(obj, maxkeylen=12, sep='/', indent=0) -> None:
  width = maxkeylen+len(sep)
  startline = False
  if isinstance(obj, uproot.rootio.ROOTDirectory):
    print('TFile: '+obj.name.decode('utf-8'))
    startline = True
    indent = 2
  elif issubclass(type(obj), uproot.tree.TTreeMethods):
    print('TTree: '+obj.name.decode('utf-8'))
    startline = True
    indent = 4
  else:
    if len(obj.keys()) > 0:
      indent += width
      s = obj.name.decode('utf-8')[:maxkeylen]
      print(s + ' '*(maxkeylen-len(s)) + sep, end='')
    else:
      print(obj.name.decode('utf-8'))

  if len(obj.keys()) > 0:
    for i, key in enumerate(obj.keys()):
      if i>0 or startline:
        print(' '*indent, end='')
      show_uproot_tree(obj[key], indent=indent)
    indent -= width

In [54]:
filepath = os.path.join(datadir, 'farichsim_1200kevt.root')
show_uproot_tree(uproot.open(filepath))

TFile: ./farichsim_pi-pi-_45-360deg_1200.0k_ideal_2020-12-24_rndm.root
  TTree: info_sim
    info_gen    /m_num_events
                 m_z_dis
    info_rad    /m_layers    /m_layers.first
                              m_layers.second
    info_pmt    /m_name
                 m_num_side_x
                 m_num_side_y
                 m_gap
                 m_size
                 m_chip_num_size
                 m_chip_pitch
                 m_chip_size
                 m_chip_offset
                 m_focal_length
                 m_trg_window
                 m_origin_pos/m_origin_pos._2
                              m_origin_pos._1
                              m_origin_pos._0
  TTree: raw_data
    event       /m_id_event
                 m_id_primary
                 m_pos_primar/m_pos_primary._2
                              m_pos_primary._1
                              m_pos_primary._0
                 m_dir_primar/m_dir_primary._2
                              m_dir_primary._1


In [55]:
def readInfoFromRoot(filepath, verbose: bool = False) -> pd.DataFrame:
  '''
  Получение информации о моделировании из ROOT-файла в виде датафрейм формой (1, N), где N - число параметров.
  '''
  # Названия используемых колонок данных для переименования и сохранения в data frame
  idf_rename_map = {'m_num_events': 'nevents',  # число событий моделирования
                    'm_z_dis': 'zdis',  # расстояние от места рождения частицы до входа в радиатор в мм
                    'm_layers': 'nlayers',  # число слоев радиатора
                    'm_size': 'array_size',  # размер матрицы КФУ в мм
                    'm_gap': 'array_gap',  # зазор между матрицами КФУ в мм
                    'm_chip_size': 'pixel_size',  # размер пикселя КФУ в мм
                    'm_chip_pitch': 'pixel_gap',  # зазор между пикселями КФУ в мм
                    'm_chip_num_size': 'pixel_numx',  # размер матрицы КФУ в пикселях
                    'm_num_side_x': 'nxarrays', 'm_num_side_y': 'nyarrays',  # размер фотодетектора в матрицах КФУ по X и Y
                    'm_focal_length': 'distance',  # расстояние от входа в радиатор до входа в фотодетектор
                    'm_trg_window': 'trg_window_ns',  # размер временного окна в нс
                    'W': 'W',  # толщина радиатора в мм (вычисляемая)
                    'n_mean': 'n_mean',  # средний показатель преломления радиатора (вычисляемый)
                    'n_max': 'n_max',  # максимальный показатель преломления радиатора (вычисляемый)
                   }

  # Открытие ROOT-файла с данными используя Uproot https://github.com/scikit-hep/uproot3
  with uproot.open(filepath) as file:
    idf = file[b'info_sim'].pandas.df('*', flatten=False)

  # Переименование параметров
  idf.rename(columns=idf_rename_map, inplace=True, errors='ignore')

  # Получение параметров (многослойного) радиатора одинаковых для всех файлов
  n_l = idf.at[0,'m_layers.first']  # показатели преломления слоёв
  w_l = idf.at[0,'m_layers.second']  # толщины слоёв радиатора

  W = w_l.sum()  # суммарная толщина всех слоёв
  n_mean = n_l.mean()  # средний показатель преломления
  n_max = n_l.max()  # максимальный показатель преломления

  # Добавление вычисляемых параметров в idf
  idf['W'] = W
  idf['n_mean'] = n_mean
  idf['n_max'] = n_max

  # Сохранение нужных параметров
  idf = idf[idf_rename_map.values()]

  if verbose:
    for name in idf.columns:
      print(f'{name}: {idf.at[0, name]}')

  return idf

In [56]:
idf = readInfoFromRoot(filepath)

In [57]:
idf

,nevents,zdis,nlayers,array_size,array_gap,pixel_size,pixel_gap,pixel_numx,nxarrays,nyarrays,distance,trg_window_ns,W,n_mean,n_max
entry,,,,,,,,,,,,,,,
0,1200000,1.0,4,26.68,1.0,3.16,0.2,8,30,30,200.0,20.0,35.0,1.0454,1.05


In [58]:
def genChunkFromRoot(filepath, eventchunksize=2000, noisefreqpersqmm: float = 2e6, noiseTimeRange: float = (0, 7), shiftSignalTimes: bool = True,
                     edfstore: pd.HDFStore = None, verbose: bool = True) -> pd.DataFrame:
  """
    Генератор событий из ROOT-файла в виде датафрейма. Число событий eventchunksize, читаемых генератором за один раз, должен выбираться так,
    чтобы все данные с учетом добавляемых шумовых срабатываний умещались в размер ОЗУ.

    Параметры:
    filepath - путь к ROOT-файлу для чтения.
    eventchunksize - число событий, загружаемых из ROOT-файла за один вызов.
    noisefreqpersqmm - частота темновых срабатываний на единицу активной площади фотодетектора в с^{-1}*мм^{-2}, подмешиваемых к событиям;
                       максимальное значение параметра, которое имеет смысл рассматривать, 2e6.
    noiseTimeRange - (start, stop) - tuple, задающий временной интервал генерации шума в наносекундах.
    edfstore - HDF-хранилище для записи датафрейма "edf"; данные добавляются к уже записанным в хранилище.
    verbose - флаг отладочной печати.

    Описание условий моделирования:
    Ось Z направлена по нормали к плоскости радиатора от радиатора к фотодетектору.
    Оси X и Y паралельны осям симметрии матрицы фотодетектора.
    Первичная частица (отрицательный пион) вылетает на расстоянии zdis=1 мм перед радиатором в его сторону
    Начальное положение частицы случайно разбрасывается по X и Y в квадрате со стороной (pixel_size+pixel_gap).
    Направление частицы случайно разбрасывается в телесном угле в пределах theta_p=[0, π/4], phi_p=[0, 2π].
    Скорость частицы случайно и равномерно разбрасывается от 0.957 до 0.999 скорости света.
  """
  global rng

  # Данные о частице (для переименования и сохранения)
  part_rename_map = {'m_hits': 'nhits',                # число срабатываний в событии
                     'm_pos_primary._0': 'x_p',        # X-координата вылета частицы в мм
                     'm_pos_primary._1': 'y_p',        # Y-координата вылета частицы в мм
                     'm_pos_primary._2': 'z_p',        # Z-координата вылета частицы в мм
                     'm_dir_primary._0': 'nx_p',       # X-компонента единичного вектора направления частицы
                     'm_dir_primary._1': 'ny_p',       # Y-компонента единичного вектора направления частицы
                     'm_dir_primary._2': 'nz_p',       # Z-компонента единичного вектора направления частицы
                     'm_beta_primary': 'beta',         # скорость частицы в единицах скорости света
                     'm_theta_primary': 'theta_p',     # полярный угол направления частицы в радианах
                     'm_phi_primary': 'phi_p',         # азимутальный угол направления частицы в радианах
                     'm_momentum_primary': 'momentum'  # импульс частицы в МэВ/c
                    }
  
  # Наблюдаемые данные о срабатываниях (для переименования и сохранения)
  hit_rename_map = {'m_hits.m_photon_pos_chip._0': 'x_c',  # X-координата срабатывания в мм
                    'm_hits.m_photon_pos_chip._1': 'y_c',  # Y-координата срабатывания в мм
                    'm_hits.m_photon_pos_chip._2': 'z_c',  # Z-координата срабатывания в мм
                    'm_hits.m_photon_time': 't_c'          # время срабатывания в нс
                   }

  other_rename_map = {'m_id_event': '',                         #
                      'm_id_primary': '',                       #
                      'm_hits.m_photon_id_pmt': '',             #
                      'm_hits.m_photon_id_chip': '',            #
                      'm_hits.m_photon_id_layer': '',           #
                      'm_hits.m_photon_id_track': '',           #
                      'm_hits.m_photon_id_track_parent': '',    #
                      'm_hits.m_photon_id_hit': '',             #
                      'm_hits.m_photon_wl': '',                 #
                      'm_hits.m_photon_pos_exact._0': '',       #
                      'm_hits.m_photon_pos_exact._1': '',       #
                      'm_hits.m_photon_pos_exact._2': '',       #
                      'm_hits.m_photon_pos_vertex._0': '',      #
                      'm_hits.m_photon_pos_vertex._1': '',      #
                      'm_hits.m_photon_pos_vertex._2': '',      #
                      'm_hits.m_photon_dir_vertex._0': '',      #
                      'm_hits.m_photon_dir_vertex._1': '',      #
                      'm_hits.m_photon_dir_vertex._2': '',      #
                      'm_hits.m_hit_is_fittable': '',           #
                      'm_hits.m_photon_theta.first': '',        #
                      'm_hits.m_photon_theta.second': '',       #
                      'm_hits.m_photon_phi.first': '',          #
                      'm_hits.m_photon_phi.second': '',         #
                      }

  for key in other_rename_map:
    other_rename_map.update({key: key})
  # Наименования колонок для сохранения в датафрейм
  edfcolstosave = list(part_rename_map.values()) + list(hit_rename_map.values())

  # Чтение параметров моделирования
  idf = readInfoFromRoot(filepath)

  # Определения параметров фотодетектора для генерации темнового шума
  pixel_size, pixel_gap = idf.at[0, 'pixel_size'], idf.at[0, 'pixel_gap']
  array_size, array_gap = idf.at[0, 'array_size'], idf.at[0, 'array_gap']
  nxpixels_arr = idf.at[0, 'pixel_numx']
  nxpixels_tot = idf.at[0, 'nxarrays']*nxpixels_arr
  igrid = np.arange(nxpixels_tot//2)
  xpnts = array_gap/2 + (igrid//nxpixels_arr)*(array_size+array_gap) + (igrid%nxpixels_arr)*(pixel_size+pixel_gap) + pixel_size/2
  xpnts = np.sort(np.append(-xpnts, xpnts)).astype('float32')
  xgrid, ygrid = np.meshgrid(xpnts, xpnts)
  xgrid = xgrid.reshape(xgrid.size)
  ygrid = ygrid.reshape(ygrid.size)
 
  def addNoise(partdf: pd.DataFrame, hitdf: pd.DataFrame) -> pd.DataFrame:
    ''' 
    Генерация темновых срабатываний темнового шума и добавление в датафрейм (без учета "мёртвого" времени пикселя).
    partdf - датафрейм для частиц
    hitdf - датафрейм для срабатываний
    '''
    assert(np.isclose(pixel_size*nxpixels_arr+pixel_gap*(nxpixels_arr-1), array_size))
    nevents = partdf.shape[0]  # число событий
    
    # среднее число шумовых срабатываний на событие
    munoise = (noiseTimeRange[1]-noiseTimeRange[0])*1e-9*noisefreqpersqmm*(pixel_size**2)*(nxpixels_tot**2)

    print(f'    Generate noise with DCR per mm^2 {noisefreqpersqmm}, mean number of hits per event: {munoise:.2f}.')

    noisehits = rng.poisson(munoise, nevents)   # генерация массива числа шумовых срабатываний в событиях по пуассоновскому распределению
    Ndc = int(noisehits.sum())                  # общее число шумовых срабатываний (скаляр)
    signalhits = partdf['nhits'].to_numpy()     # массив числа сигнальных срабатываний по событиям

    # случайное смещение сигнальных срабатываний в пределах временного окна генерации шума
    if shiftSignalTimes:
      hitdf['t_c'] += np.repeat(rng.uniform(0, noiseTimeRange[1]-2, nevents), partdf['nhits'])

    hitdf['signal'] = np.ones(signalhits.sum(), bool)  # разметка сигнальных срабатываний значением 'signal' True
    if Ndc == 0:    # если нет шумовых срабатываний
      return hitdf  # возвращаем исходный датафрейм с добавлением колонки 'signal'

    ich = rng.choice(xgrid.size, Ndc)           # генерация случайных номеров сработавших каналов с возможным повтором
    xh = xgrid[ich]                             # x-координата сработавших каналов
    yh = ygrid[ich]                             # y-координата сработавших каналов
    # zh = hitdf.at[(0, 0), 'z_c']                # z-координата срабатываний (скаляр)
    zh = 201.050003
    th = rng.uniform(noiseTimeRange[0], noiseTimeRange[1], size=Ndc) # генерация времён срабатываний по однородному распределению
   
    # нумерация шумовых срабатываний по событиям
    ievent = np.repeat(partdf.index, noisehits) # массив номеров событий для записи в датафрейм
    ihit = np.zeros(Ndc, 'int64')               # инициализация массива номеров срабатываний для записи в датафрейм
    index = 0
    for i in range(nevents):
      ihit[index:index+noisehits[i]] = signalhits[i] + np.arange(noisehits[i])
      index += noisehits[i]
    
    # создание датафрейма с шумовыми срабатываниями того же формата, что hitdf
    noisedf = pd.DataFrame({'x_c': xh, 'y_c': yh, 'z_c': zh, 't_c': th, 'signal': np.zeros(Ndc, bool)},
                           index=pd.MultiIndex.from_arrays((ievent, ihit), names=('entry', 'subentry')))

    # TO DO: случайное смещение кольца в фотодетекторе (сдвиг координат сигнальных хитов).
    # Сложность с реализацией для неравномерной сетки пикселей, т.к. зазоры между матрицами больше зазоров между пикселями в матрице.
    # Проще сделать в моделировании.

    # сливаем сигнальный и шумовой датафрейм и сортируем указатель событий и срабатываний
    hitdf2 = pd.concat((hitdf, noisedf), copy=False).sort_index(level=('entry', 'subentry'))

    # обновляем количества срабатываний в partdf, добавляя количества шумовых срабатываний по событиям
    partdf['nhits'] += noisehits

    return hitdf2


  nFileEvents = idf.at[0, 'nevents']
  # print(f'Processing ROOT file {filepath} with {nFileEvents} simulated events...', flush=True)
  
  # Цикл чтения кусков ROOT-файла
  for partdf, hitdf, otherdf in zip(uproot.pandas.iterate(filepath, "raw_data", part_rename_map.keys(), entrysteps=eventchunksize),
                           uproot.pandas.iterate(filepath, "raw_data", hit_rename_map.keys(), entrysteps=eventchunksize, flatten=True),
                           uproot.pandas.iterate(filepath, "raw_data", other_rename_map.keys(), entrysteps=eventchunksize, flatten=True)):
    # print('\n  Processing next chunk...')

    # Переименование колонок
    partdf.rename(columns=part_rename_map, inplace=True, errors='raise')
    hitdf.rename(columns=hit_rename_map, inplace=True, errors='raise')
    otherdf.rename(columns=other_rename_map, inplace=True, errors='raise')

    partdf = partdf.astype('float32', copy=False)
    partdf['nhits'] = partdf['nhits'].astype('int32', copy=False)
    hitdf = hitdf.astype('float32', copy=False)
    otherdf = otherdf.astype('float32', copy=False)

    # Генерация и добавление шумовых срабатываний
    hitdf = addNoise(partdf, hitdf)

    # print(f'    {hitdf.index.levels[0].size} entries with {hitdf.shape[0]} hits to process')

    # Слияние данных событий и срабатываний
    edf = hitdf.join(otherdf)
    edf = edf.join(partdf, on='entry')
    #edf = hitdf.join(partdf, on='entry')
    #edf = edf.join(otherdf, on='entry')
    if verbose:
      pass
      # print(edf)

    if edfstore is not None:
      # print(f'    Saving edf chunk...')
      edfstore.put('edf', edf, format='table', append=True)

    yield edf

In [59]:
# Чтение и домоделирование одного "куска" данных
#edf = next(genChunkFromRoot(filepath, 1450, noisefreqpersqmm=1000000))
edf = next(genChunkFromRoot(filepath, 10000, noisefreqpersqmm=0))

    Generate noise with DCR per mm^2 0, mean number of hits per event: 0.00.


In [60]:
def edf_to_bdf(edf_col: pd.Series, bdf: pd.DataFrame):
  to_bdf = [sub.iloc[0] for _, sub in edf_col.groupby(level=0)]
  bdf[edf_col.name] = pd.Series(to_bdf)

In [61]:
edf

x_c         y_c         z_c       t_c  signal   
entry subentry                                                         
0     0         -70.879997 -226.880005  201.050003  2.188593    True  \
      1         -53.279999 -309.920013  201.050003  2.396409    True   
      2         -36.480000 -306.559998  201.050003  2.382249    True   
      3         -49.919998 -184.960007  201.050003  2.073419    True   
      4         -12.160000 -288.959991  201.050003  2.326930    True   
...                    ...         ...         ...       ...     ...   
9999  40        178.240005  -80.959999  201.050003  3.695349    True   
      41        178.240005  -74.239998  201.050003  3.687293    True   
      42        226.880005  -53.279999  201.050003  3.892318    True   
      43        274.720001 -209.279999  201.050003  4.102964    True   
      44        251.199997  -18.879999  201.050003  3.840951    True   

                m_id_event  m_id_primary  m_hits.m_photon_id_pmt   
entry subentry                                                     
0     0                1.0        -211.0                   367.0  \
      1                1.0        -211.0                   394.0   
      2                1.0        -211.0                   394.0   
      3                1.0        -211.0                   399.0   
      4                1.0        -211.0                   425.0   
...                    ...           ...                     ...   
9999  40           10000.0        -211.0                   643.0   
      41           10000.0        -211.0                   643.0   
      42           10000.0        -211.0                   704.0   
      43           10000.0        -211.0                   728.0   
      44           10000.0        -211.0                   735.0   

                m_hits.m_photon_id_chip  m_hits.m_photon_id_layer  ...   
entry subentry                                                     ...   
0     0                            31.0                       4.0  ...  \
      1                             7.0                       4.0  ...   
      2                            48.0                       4.0  ...   
      3                            11.0                       2.0  ...   
      4                            37.0                       3.0  ...   
...                                 ...                       ...  ...   
9999  40                           25.0                       1.0  ...   
      41                           27.0                       3.0  ...   
      42                            9.0                       4.0  ...   
      43                           60.0                       4.0  ...   
      44                            3.0                       4.0  ...   

                     x_p       y_p  z_p      nx_p      ny_p      nz_p   
entry subentry                                                          
0     0         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977  \
      1         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      2         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      3         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      4         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
...                  ...       ...  ...       ...       ...       ...   
9999  40        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      41        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      42        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      43        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      44        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   

                    beta   theta_p     phi_p    momentum  
entry subentry                                            
0     0         0.966818  0.774205  4.607758  528.209290  
      1         0.966818  0.774205  4.607758  528.209290  
      2         0.966818  0.774205  4.607758  528.20

In [62]:
def recoAngles(edf: pd.DataFrame, idf: pd.DataFrame, rotation_mode = False):
  '''
  Геометрическая реконструкция углов фотонов относительно направления частицы.
  Из координат срабатываний и частиц вычисляются углы theta_c, phi_c и время вылета фотонов t_c_orig и добавляются к edf.
  '''
  r0 = edf.loc[:, ('x_p', 'y_p', 'z_p')].to_numpy()
  if rotation_mode:
    r = edf.loc[:, ('rotated_x', 'rotated_y', 'rotated_z')].to_numpy()
    # n0 = edf.loc[:, ('rotated_nx_p', 'rotated_ny_p', 'rotated_nz_p')].to_numpy()
    n0 = edf.loc[:, ('recalculated_nx_p', 'recalculated_ny_p', 'recalculated_nz_p')].to_numpy()
  else:
    r  = edf.loc[:, ('x_c', 'y_c', 'z_c')].to_numpy()
    n0 = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()

  speedOfLight_mmperns = 299.792458 # мм/нс

  # расстояние от радиатора до детектора
  dist = float(idf['distance'])

  # толщина радиатора
  W = float(idf['W'])

  # расстояние от точки вылета частицы до входной плоскости радиатора
  rad_pos = float(idf['zdis'])

  # полное число срабатываний
  N = edf.shape[0]

  # координаты точки пересечения трека с ФД
  if not rotation_mode:
    y_i = r0[:,1] + (dist + rad_pos) * n0[:,1] / n0[:,2] # r0[:,1] + (dist + W + rad_pos) * n0[:,1] / n0[:,2]   #   r0[:,1] + (dist + rad_pos) * n0[:,1] / n0[:,2]
    x_i = r0[:,0] + (y_i - r0[:,1]) * n0[:,0] / n0[:,1] # r0[:,0] + (y_i - r0[:,1]) * n0[:,0] / n0[:,1]    #     r0[:,0] + (dist + rad_pos) * n0[:,0] / n0[:,2]
    edf['x_i'] = x_i
    edf['y_i'] = y_i
    edf['r_p_c'] = np.sqrt((r0[:,0] - x_i) ** 2 + (r0[:,1] - y_i) ** 2 + (r0[:,2] - r[:,2]) ** 2)
    edf['r_c'] = np.sqrt((x_i - edf['x_c']) ** 2 + (y_i - edf['y_c']) ** 2)

  if rotation_mode:
    n_mean = float(idf['n_mean'])

    edf['rotated_r_c'] = np.sqrt((edf['rotated_x_i'] - edf['rotated_x']) ** 2 + (edf['rotated_y_i'] - edf['rotated_y']) ** 2)

    rotated_r_c = edf['rotated_r_c'].to_numpy()
    # r_p_c = edf['r_p_c'].to_numpy()
    beta = edf['beta'].to_numpy()
    r_p_c = dist # or + W/2 ???

    edf['beta_from_true_r'] = np.sqrt(rotated_r_c ** 2 + r_p_c ** 2) / (n_mean * r_p_c)
    edf['true_r_from_beta'] = r_p_c * np.sqrt((n_mean * beta) ** 2 - 1)

    avg_betas = []
    for _, subentry in edf['beta_from_true_r'].groupby(level=0):
      avg_beta = subentry.mean()
      for __ in subentry:
        avg_betas.append(avg_beta)
    edf['beta_from_true_r_mean'] = avg_betas
  # косинусы и синусы сферических углов направления частицы
  costheta, sintheta = n0[:,2], np.sqrt(n0[:,0]**2+n0[:,1]**2)
  phi = np.arctan2(n0[:,1], n0[:,0])
  cosphi, sinphi = np.cos(phi), np.sin(phi)

  # номинальная точка вылета фотонов
  ro = r0 + (W/2+rad_pos)/n0[:,2].reshape(N,1)*n0

  """
  Преобразование в СК частицы
  𝑢𝑥 = cos 𝜃(𝑣𝑥 cos 𝜙 + 𝑣𝑦 sin 𝜙) − 𝑣𝑧 sin 𝜃,
  𝑢𝑦 = −𝑣𝑥 sin 𝜙 + 𝑣𝑦 cos 𝜙,
  𝑢𝑧 = sin 𝜃(𝑣𝑥 cos 𝜙 + 𝑣𝑦 sin 𝜙) + 𝑣𝑧 cos 𝜃.
  """

  # вектор направления фотона в лабораторной СК
  s = (r-ro)
  snorm = np.linalg.norm(s, axis=1, keepdims=True)
  v = s / snorm
  if not rotation_mode:
    edf['t_c_orig'] = edf['t_c'] - (snorm / speedOfLight_mmperns).reshape(N)

  # освобождение памяти при необходимости
  #del r0, n0, ro, r, s

  U = np.stack((np.stack((costheta*cosphi, costheta*sinphi, -sintheta)),
                np.stack((-sinphi,         cosphi,          np.full(N, 0.))),
                np.stack((sintheta*cosphi, sintheta*sinphi, costheta)))).transpose(2,0,1)

  # единичный вектор направления фотона в СК частицы
  u = (U @ v.reshape(N,3,1)).reshape(N,3)

  # сферические углы фотона в СК частицы
  if rotation_mode:
    edf['rotated_theta_c'] = np.arccos(u[:,2])
    edf['rotated_phi_c'] = np.arctan2(-u[:,1], -u[:,0])
  else:
    edf['theta_c'] = np.arccos(u[:,2])
    edf['phi_c'] = np.arctan2(-u[:,1], -u[:,0])
    avg_thetas = []
    for _, subentry in edf['theta_c'].groupby(level=0):
      avg_theta = subentry.mean()
      for __ in subentry:
        avg_thetas.append(avg_theta)
    edf['theta_c_mean'] = avg_thetas

In [63]:
def applySpaceCut(edf: pd.DataFrame) -> pd.DataFrame:
  return edf[(abs(edf['x_c'] - edf['x_i']) <= 220) & (abs(edf['y_c'] - edf['y_i']) <= 220)]

In [64]:
def signalLength(edf: pd.DataFrame):
  t_signal = edf[edf['signal'] ==  True]['t_c']
  t = []
  for _, subentry in t_signal.groupby(level=0):
    t_min = np.min(subentry)
    t_max = np.max(subentry)
    t.extend(t_max - t_min for __ in subentry)
  t = np.array(t)
  edf['signal_length'] = t

In [65]:
def planeRecalculation(edf: pd.DataFrame, idf: pd.DataFrame):
    R_p = edf[['x_p', 'y_p', 'z_p']].to_numpy()
    R = edf[['x_c', 'y_c', 'z_c']].to_numpy()
    R_i = edf[['x_i', 'y_i', 'z_c']].to_numpy()
    N = edf[['nx_p', 'ny_p', 'nz_p']].to_numpy()
    dist = idf.W / 2 + idf.zdis
    alpha = (float(dist) - R_p[:,2]) / N[:,2]
    r_d = R_p + N * alpha[:, np.newaxis]

    u = R - r_d
    dot = np.sum(N * u, axis=1)
    w = r_d - R_i
    fac = -np.sum(N * w, axis=1) / dot
    u *= fac[:, np.newaxis]

    R_new = r_d + u



    edf['recalculated_x'] = R_new[:,0]
    edf['recalculated_y'] = R_new[:,1]
    edf['recalculated_z'] = R_new[:,2]

In [66]:
def primaryDirectionRecalculation(edf: pd.DataFrame):
  N = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()
  M = []
  for n in N:
    # C = np.stack((np.array([-(n[1] ** 2 + n[2] ** 2) / n[0], 0 , n[0]]),
    #               np.array([n[1], - n[2], n[1]]),
    #               np.array([n[2], n[1], n[2]])))

    # C_inv = np.array([np.array([- n[0] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[0] ** 2 * n[1] / (n[1] ** 4 + n[0] ** 2 * n[1] ** 2 + n[2] ** 4 + n[0] ** 2 * n[2] ** 2 + 2 * n[1] ** 2 * n[2] ** 2) , n[0] ** 2 * n[2] / (n[1] ** 4 + n[0] ** 2 * n[1] ** 2 + n[2] ** 4 + n[0] ** 2 * n[2] ** 2 + 2 * n[1] ** 2 * n[2] ** 2)]),
    #               np.array([0, - n[2] / (n[1] ** 2 + n[2] ** 2), n[1] / (n[1] ** 2 + n[2] ** 2)]),
    #               np.array([n[0] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[1] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[2] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2)])])
    M.append([0, 0, 1])
    # print(n)
    # print(C_inv)
    # print(C_inv @ n)
    # break
  M = np.array(M)
  edf['recalculated_nx_p'] = M[:,0]
  edf['recalculated_ny_p'] = M[:,1]
  edf['recalculated_nz_p'] = M[:,2]

In [67]:
def planeRotation(edf: pd.DataFrame):
  R = edf[['recalculated_x', 'recalculated_y', 'recalculated_z']].to_numpy()
  R_i = edf[['x_i', 'y_i', 'z_c']].to_numpy()
  N = edf[['nx_p', 'ny_p', 'nz_p']].to_numpy() # N
  M = np.array([0, 0, 1])                           # M
  c = np.dot(N, M) / (np.linalg.norm(M) * np.linalg.norm(N, axis=1))
  axis = np.cross(N, np.broadcast_to(M, (N.shape[0], 3))) / np.linalg.norm(np.cross(N, np.broadcast_to(M, (N.shape[0], 3))), axis=1, keepdims=True)
  x, y, z = axis.T
  s = np.sqrt(1-c*c)
  C = 1-c
  rmat = np.array([
      [x*x*C+c, x*y*C-z*s, x*z*C+y*s],
      [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
      [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])
  # print(rmat.shape)
  # print(R.shape)
  # print(rmat[:, :, 0])
  # print(R[0])
  # print(rmat[:, :, 0] @ R[0])
  rotated_R = np.matmul(rmat.transpose((2, 0, 1)), R[:, :, np.newaxis])
  rotated_R = np.squeeze(rotated_R, axis=-1).transpose().T
  rotated_R_i = np.matmul(rmat.transpose((2, 0, 1)), R_i[:, :, np.newaxis])
  rotated_R_i = np.squeeze(rotated_R_i, axis=-1).transpose().T
  # print(rotated_R[0])
  maskR = np.logical_or(abs(rotated_R[:, 0]) >= 500, abs(rotated_R[:, 1]) >= 500)
  maskR_i = np.logical_or(abs(rotated_R_i[:, 0]) >= 500, abs(rotated_R_i[:, 1]) >= 500)
  rotated_R[maskR] = [5000, 5000, 0]
  rotated_R_i[maskR_i] = [5000, 5000, 0]
  rotated_n = (rotated_R_i - edf[['x_p', 'y_p', 'z_p']].to_numpy()) / np.linalg.norm(rotated_R_i - edf[['x_p', 'y_p', 'z_p']].to_numpy(), axis=1, keepdims=True)
  edf['rotated_x'] = rotated_R[:,0]
  edf['rotated_y'] = rotated_R[:,1]
  edf['rotated_z'] = rotated_R[:,2]
  edf['rotated_x_i'] = rotated_R_i[:,0]
  edf['rotated_y_i'] = rotated_R_i[:,1]
  edf['rotated_z_i'] = rotated_R_i[:,2]
  edf['rotated_nx_p'] = rotated_n[:,0]
  edf['rotated_ny_p'] = rotated_n[:,1]
  edf['rotated_nz_p'] = rotated_n[:,2]

In [68]:
def applySecondSpaceCut(edf: pd.DataFrame) -> pd.DataFrame:
  return edf[(abs(edf['rotated_x'] - edf['rotated_x_i']) <= 220) & (abs(edf['rotated_y'] - edf['rotated_y_i']) <= 220)]

In [69]:
def local_sum_2d(event, r_slices, t_slices, square_counts, max_index, n, m, timestep, t_window_width, method='N/r'):
  # cut_event = event[(event.t_c <= t_slices[np.clip(max_index[1] + m, a_min=0, a_max=10)]) & (event.t_c >= t_slices[np.clip(max_index[1] - m, a_min=0, a_max=10)]) &
  #                   (event.rotated_r_c <= r_slices[max_index[0] + n]) & (event.rotated_r_c >= r_slices[max_index[0] - n])]
  cut_event = event[(event.t_c <= np.clip(t_slices[max_index[1]] + t_window_width + timestep * m, 0, 10)) & (event.t_c >= np.clip(t_slices[max_index[1]] - timestep * m, 0, 10)) &
                    (event.rotated_r_c <= r_slices[max_index[0] + n]) & (event.rotated_r_c >= r_slices[max_index[0] - n])]
  return np.mean(cut_event.rotated_r_c)

In [70]:
def local_weighed_sum_2d(r_slices, t_slices, square_counts, max_index, n, m, method='N/r'):
  arr = np.mean(square_counts[max_index[0] - n:max_index[0] + n + 1, np.clip(max_index[1] - m, 0, 50):np.clip(max_index[1] + m + 1, 0, 50)], axis=1)
  if method == 'N/r':
    sum_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] ** 2 * arr
    den_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] * arr
  else:
    sum_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] * arr
    den_arr = arr

  weighted_sum = np.sum(sum_arr)
  weighted_den = np.sum(den_arr)

  return weighted_sum / weighted_den

In [71]:
def local_weighed_sum(r_slices, counts, max_index, n, method='N/r'):
  if method == 'N/r':
    sum_arr = r_slices[max_index - n:max_index + n + 1] ** 2 * counts[max_index - n:max_index + n + 1]
    den_arr = r_slices[max_index - n:max_index + n + 1] * counts[max_index - n:max_index + n + 1]
  else:
    sum_arr = r_slices[max_index - n:max_index + n + 1] * counts[max_index - n:max_index + n + 1]
    den_arr = counts[max_index - n:max_index + n + 1]

  weighted_sum = np.sum(sum_arr)
  weighted_den = np.sum(den_arr)

  return weighted_sum / weighted_den

In [72]:
def pol(x, a, b, c):
  return a * np.exp((x - b) ** 2 / c ** 2)

In [73]:
def rSlidingWindowIntro(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=False, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, num_of_groups=5):
  r_r_c = edf['rotated_r_c']
  time_step = float(t_window_width) / t_width_factor
  all_avgs = np.array(r_r_c.groupby(level=0).transform('mean').to_list()).ravel()
  all_dists = np.abs(r_r_c - all_avgs)
  all_sigms = np.array(r_r_c.groupby(level=0).transform('std').to_list()).ravel()

  edf['mean_rotated_r_c'] = all_avgs
  edf['dist_from_mean_rotated_r_c'] = all_dists
  edf['rotated_r_c_sigm'] = all_sigms

  # Compute beta_step and r_step using NumPy functions
  beta_step = np.ptp(edf['beta'].values) # не факт что нужно values
  r_step = np.ptp(edf['true_r_from_beta'].values)

  # Compute beta_intervals using NumPy linspace function
  num_of_groups = num_of_groups
  beta_intervals = np.linspace(edf['beta'].min(), edf['beta'].max(), num=num_of_groups)

  # Compute beta_group_to_bdf and true_r_group using NumPy operations
  beta_group = np.floor((num_of_groups * edf['beta'] + max(edf['beta']) - (num_of_groups + 1) * min(edf['beta'])) / beta_step).values
  true_r_group = np.floor((num_of_groups * edf['true_r_from_beta'] + max(edf['true_r_from_beta']) - (num_of_groups + 1) * min(edf['true_r_from_beta'])) / r_step).values

  edf['beta_group'] = beta_group
  edf['true_r_group'] = true_r_group

  edf_to_bdf(edf.beta_group, bdf)
  edf_to_bdf(edf.true_r_group, bdf)

  edf_to_bdf(edf.theta_p, bdf)
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  # edf_to_bdf(edf.signal_counts, bdf)
  edf_to_bdf(edf.beta, bdf)
  edf_to_bdf(edf.true_r_from_beta, bdf)

  # edf['slice_group'] = np.floor(r_r_c / step) * step
  edf['slice_group'] = r_r_c # test
  # edf['time_slice_group'] = np.floor(edf.t_c / time_step) * time_step
  edf['time_slice_group'] = edf.t_c # test2


In [74]:
def calculateSignalCounts(edf: pd.DataFrame, bdf: pd.DataFrame):
  signal_counts = edf['signal'].groupby(level=0).sum()
  bdf['signal_counts'] = signal_counts.values
  edf['signal_counts'] = edf.signal.groupby(level=0).transform('sum').values

In [75]:
def rSlidingWindowLoop1(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=False, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = True, weighed = True):
  step = step / r_width_factor
  time_step = float(t_window_width) / t_width_factor

  r_slices = np.arange(0, 800, step=step)
  t_slices = np.arange(0, 15, step=time_step)

  n_sigmas = np.ptp(avg_sigmas)
  t_sigmas = np.ptp(avg_t_sigmas)
  # all_counts_to_edf = np.zeros((n_sigmas, len(edf)))
  all_counts_to_edf = np.zeros((n_sigmas, len(edf)))
  all_calculated_r = np.zeros((n_sigmas, len(edf)))
  all_calculated_r_from_2d = np.zeros((t_sigmas, n_sigmas, len(edf)))
  cur_ind = 0
  for i, (entry, subentry) in enumerate(edf[['slice_group', 'time_slice_group', 'rotated_r_c', 't_c']].groupby(level=0)):
    counts = np.zeros(r_slices.shape)
    square_counts = np.zeros(shape=(r_slices.shape[0], t_slices.shape[0]))

    mask = np.logical_and(subentry.slice_group >= 16, subentry.slice_group <= 80)
    slice_groups = subentry.slice_group[mask]
    time_slice_groups = subentry.time_slice_group[mask]

    counts, _ = np.histogram(slice_groups, bins=r_slices)
    square_counts, _, __ = np.histogram2d(slice_groups, time_slice_groups, bins=(r_slices, t_slices))

    if method == 'N/r':
      counts = np.divide(np.add(counts[:-1], counts[1:]), r_slices[1:-1])
      square_counts[:-1, :] = np.divide(np.add(square_counts[:-1, :], square_counts[1:, :]), r_slices[1:-1, np.newaxis])
    if full_width_t_hist:
      square_counts_but_last = sum([square_counts[:, it : -t_width_factor + 1 + it] for it in range(t_width_factor - 1)])
      square_counts = np.add(square_counts_but_last, square_counts[:, t_width_factor - 1:])

    max_index = np.argmax(counts)

    max_index_2d = np.unravel_index(np.argmax(square_counts), square_counts.shape)

    for j in range(n_sigmas):
      all_counts_to_edf[j][cur_ind:subentry.shape[0] + cur_ind] = counts[np.floor_divide(subentry.slice_group, step).astype(int)] # fixed
      # avg_r_from_slices = local_weighed_sum(r_slices, counts, max_index, j + avg_sigmas[0], method)
      # all_calculated_r[j, cur_ind:subentry.shape[0] + cur_ind] = np.repeat(avg_r_from_slices, subentry.shape[0])
      for t in range(t_sigmas):
        if weighed:
          avg_r_from_2d_slices = local_weighed_sum_2d(r_slices, t_slices, square_counts, max_index_2d, j + avg_sigmas[0], t + avg_t_sigmas[0])
        else:
          avg_r_from_2d_slices = local_sum_2d(subentry, r_slices, t_slices, square_counts, max_index_2d, j + avg_sigmas[0], t + avg_t_sigmas[0], t_window_width=t_window_width, timestep=time_step)

        # if np.isnan(avg_r_from_2d_slices):
        #   print(max_index_2d)
        #   print(square_counts[max_index_2d[0] - 1:max_index_2d[0] + 2, max_index_2d[1] - 1: max_index_2d[1] + 2])
        #   raise ValueError
        all_calculated_r_from_2d[t, j, cur_ind:subentry.shape[0] + cur_ind] = np.repeat(avg_r_from_2d_slices, subentry.shape[0])

    cur_ind += subentry.shape[0]
  for j in range(n_sigmas):
    edf[f'slice_counts_{j + avg_sigmas[0]}_sigms'] = all_counts_to_edf[j]
    # edf[f'unfixed_calculated_r_{j + avg_sigmas[0]}_sigms'] = all_calculated_r[j, :]
    for t in range(t_sigmas):
      edf[f'unfixed_calculated_r_2d_{j + avg_sigmas[0]}_rsigms_{t + avg_t_sigmas[0]}_tsigms'] = all_calculated_r_from_2d[t, j, :]
      edf_to_bdf(edf[f'unfixed_calculated_r_2d_{j + avg_sigmas[0]}_rsigms_{t + avg_t_sigmas[0]}_tsigms'], bdf)

In [76]:
def rSlidingWindowLoop2(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=False, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, param_fit=False):

  # cal_arr = np.array([np.array([np.array(y) for y in x]) for x in cal_arr])

  edf_to_bdf(edf.theta_p, bdf)
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  edf['cos_theta_p'] = np.cos(edf['theta_p'])
  theta_interval = np.ptp(bdf.cos_theta_p) / 10
  theta_min = min(bdf.cos_theta_p)
  theta_max = max(bdf.cos_theta_p)
  edf_to_bdf(edf.beta, bdf)
  edf_to_bdf(edf.true_r_from_beta, bdf)

  # for n_sigms in range(*avg_sigmas):
  #   meas_betas = []
  #   for entry, subentry in edf[f'unfixed_calculated_r_{n_sigms}_sigms'].groupby(level=0):
  #     try:
  #       meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][(np.floor((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))).astype(int)]))
  #     except IndexError:
  #       meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][9]))
  #       print((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))
  #     for subsub in subentry:
  #       meas_betas.append(meas_beta)
  #   edf[f'beta_from_calc_r_{n_sigms}_sigms'] = meas_betas
  #
  #   edf[f'delta_beta_{n_sigms}_sigms'] = edf[f'beta_from_calc_r_{n_sigms}_sigms'] - edf['beta']
  #   edf[f'eps_beta_{n_sigms}_sigms'] = edf[f'delta_beta_{n_sigms}_sigms'] / edf['beta'] * 100
  #
  #   edf_to_bdf(edf[f'unfixed_calculated_r_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'delta_beta_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'eps_beta_{n_sigms}_sigms'], bdf)


  for n_sigms in range(*avg_sigmas):
    for t_sigms in range(*avg_t_sigmas):
      meas_betas = np.zeros(edf.shape[0])
      cur_ind = 0
      # meas_betas = []
      for entry, subentry in edf[f'unfixed_calculated_r_2d_{n_sigms}_rsigms_{t_sigms}_tsigms'].groupby(level=0):
        if param_fit:
          cos_theta_p = edf.cos_theta_p[entry].iloc[0]
          meas_beta = pol(subentry.iloc[0], pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][0]),
                          pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][1]),
                          pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][2]))
        else:
          if edf.cos_theta_p[entry].iloc[0] != theta_max:
            meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][(np.floor(((edf.cos_theta_p[entry].iloc[0]) - theta_min) / theta_interval)).astype(int)]))
          else:
            meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][9]))
        meas_betas[cur_ind: subentry.shape[0] + cur_ind] = np.repeat(meas_beta, subentry.shape[0])
        cur_ind += subentry.shape[0]
        # for subsub in subentry:
        #   meas_betas.append(meas_beta)  # FIXME
      edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'] = meas_betas
      edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] = edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'] - edf['beta']
      edf[f'eps_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] = edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] / edf['beta'] * 100


      edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)
      edf_to_bdf(edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)
      edf_to_bdf(edf[f'eps_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)

In [77]:
def rSlidingWindow(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=False, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = True, num_of_groups=5, weighed=True, deg_lim=False, param_fit=False):
  rSlidingWindowIntro(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, num_of_groups=num_of_groups)
  calculateSignalCounts(edf, bdf)
  rSlidingWindowLoop1(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, full_width_t_hist=full_width_t_hist, weighed=weighed)
  if cal_arr is False:
    cal_arr = np.array(calibration(edf, bdf, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, step=step, t_window_width=t_window_width,
                                   r_width_factor=r_width_factor, t_width_factor=t_width_factor, weighed=weighed, deg_lim=deg_lim, param_fit=param_fit)) # add r and t calibr - done
  rSlidingWindowLoop2(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, param_fit=param_fit)
  return cal_arr

In [78]:
def rms90(arr):
    # Calculate the mean and standard deviation of the array
    arr = arr.dropna()
    arr_mean = np.mean(arr)
    arr_std = np.std(arr)

    # Define the upper and lower limits for the 90% range
    lower_limit = np.percentile(arr, 5)
    upper_limit = np.percentile(arr, 95)
    # print(lower_limit, upper_limit)
    # print(arr)
    # Select the values within the 90% range
    arr_filtered = arr[(arr >= lower_limit) & (arr <= upper_limit)]
    # print(arr_filtered)
    assert arr_filtered.shape
    # Calculate the root mean square of the filtered values
    rms = np.std(arr_filtered)

    return rms

In [79]:
def betaGroupsRMS90(bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, n = 5):
  beta_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_epss = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_sigms_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)

  for group in range(1, n + 1):
    data = bdf[bdf['beta_group'] == group]
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        population_fourth_moment = np.mean(bdf[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        sample_fourth_moment = np.mean(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        # print(np.std(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms']))
        beta_sigms[i, j, group - 1] = rms90(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        # assert not np.isnan(beta_sigms[i, j, group - 1])
        beta_epss[i, j, group - 1] = rms90(data[f'eps_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        beta_sigms_sigms[i, j, group - 1] = np.sqrt(2 * np.abs(sample_fourth_moment - population_fourth_moment) / (data.shape[0]))

  return beta_sigms, beta_epss, beta_sigms_sigms

In [80]:
# labels = ['0', '1e3', '1e4', '1e5', '1e6']
# labels = ['DCR = ' + i + ' $Hz/mm^2$' for i in labels]
# num_of_groups = 10
# y = np.arange(1, num_of_groups + 1)
# x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups
# plt.figure(figsize=(10, 10))
# for i in range(len(labels)):
#   plt.plot(x, [0.0005] * 10, label=labels[i])
# plt.ylim((0, 0.002))
# # plt.xlim((0.954, 0.999))
# plt.legend()
# plt.grid()
# plt.show()

In [81]:
def plot_final_graph(beta_sigms, beta_sigms_yerr, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, to_save=True, deg_lim=False, num_of_groups=10):
  labels = ['0', '1e3', '1e4', '1e5', '1e6']
  labels = ['DCR = ' + i + ' $Hz/mm^2$' for i in labels]
  colors = ['c', 'y', 'g', 'r', 'm']
  weight = 'weighed' if weighed else 'unweighed'
  y = np.arange(1, num_of_groups + 1)
  x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups

  fig, axs = plt.subplots(np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), figsize=(10 * np.ptp(avg_t_sigmas), 10 * np.ptp(avg_sigmas)))
  title = f'Method: N(r) / r; {weight} Avg\nR Width = {r_width}mm, T Width = {t_width}ns\nR step factor = {r_factor}, T step factor = {t_factor}'
  if deg_lim:
    title += '\n' + r'$\theta_p < 10\deg$'
  # fig.suptitle(title)

  if np.ptp(avg_sigmas) > 1:
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        for k in range(beta_sigms.shape[0]):
          axs[i, j].plot(x, beta_sigms[k, i, j], label=labels[k], c=colors[k])
          axs[i, j].errorbar(x, beta_sigms[k, i, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
          axs[i, j].errorbar(x, beta_sigms[k, i, j], yerr=beta_sigms_yerr[k, i, j], linestyle='', c=colors[k])
        axs[i, j].legend(loc='upper right')
        axs[i, j].set_xlabel('Beta Group')
        axs[i, j].set_ylabel(r'RMS90($\Delta\beta$)')
        if deg_lim:
          axs[i, j].set_ylim((0, 0.004))
        axs[i, j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0] + i}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
        axs[i, j].grid()
  elif np.ptp(avg_t_sigmas) > 1:
    for j in range(np.ptp(avg_t_sigmas)):
      for k in range(beta_sigms.shape[0]):
        axs[j].plot(x, beta_sigms[k, 0, j], label=labels[k], c=colors[k])
        axs[j].errorbar(x, beta_sigms[k, 0, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
        axs[j].errorbar(x, beta_sigms[k, 0, j], yerr=beta_sigms_yerr[k, 0, j], linestyle='', c=colors[k])
      axs[j].legend(loc='upper right')
      axs[j].set_xlabel('Beta Group')
      axs[j].set_ylabel(r'RMS90($\Delta\beta)$')
      if deg_lim:
        axs[j].set_ylim((0, 0.004))
      axs[j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0]}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
      axs[j].grid()
  else:
    for k in range(beta_sigms.shape[0]):
      axs.plot(x, beta_sigms[k, 0, 0], label=labels[k], c=colors[k])
      axs.errorbar(x, beta_sigms[k, 0, 0], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
      axs.errorbar(x, beta_sigms[k, 0, 0], yerr=beta_sigms_yerr[k, 0, 0], linestyle='', c=colors[k])
    axs.legend(loc='upper right')
    axs.set_xlabel('Beta Group')
    axs.set_ylabel(r'RMS90($\Delta\beta$)')
    if deg_lim:
      axs.set_ylim((0, 0.002))
    axs.set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0]}$\sigma$\nt window width = {avg_t_sigmas[0]}$\sigma$')
    axs.grid()

  if to_save:
    filename = f'{weight}_avg_rw={r_width}_tw={t_width}_rs={r_factor}_ts={t_factor}_rsigms={avg_sigmas[0]}-{avg_sigmas[-1]-1}_tsigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1]-1}'
    if deg_lim:
      filename += '_10deg'
    filename += '.png'
    fig.savefig(os.path.join('results', f'{filename}'))
    plt.close(fig)
  else:
    plt.show()

In [82]:
def plot_groupped_distributions(bdf, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background, to_save=True, deg_lim=False, num_of_groups=10):
  labels = ['0', '1e3', '1e4', '1e5', '1e6']
  labels = ['DCR = ' + i + ' cps' for i in labels]
  colors = ['c', 'y', 'g', 'r', 'm']
  weight = 'weighed' if weighed else 'unweighed'
  left_lim = min(bdf[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'])
  right_lim = max(bdf[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'])
  y = np.arange(1, num_of_groups + 1)
  x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups
  x_interval_over_2 = (x[1] - x[0]) / 2
  fig, axs = plt.subplots(num_of_groups, 1, figsize=(16, 9 * num_of_groups), sharex=True)
  title = f'Method: N(r) / r; {weight} Avg\nR Width = {r_width}mm, T Width = {t_width}ns\nR step factor = {r_factor}, T step factor = {t_factor}, DCR={background}'
  if deg_lim:
    title += '\n' + r'$\theta_p < 10\deg$'
  # fig.suptitle(title)
  fig.tight_layout()
  if np.ptp(avg_sigmas) > 1:
    pass
  elif np.ptp(avg_t_sigmas) > 1:
    pass
  else:
    for group in range(1, num_of_groups + 1):
      data = bdf[bdf['beta_group'] == group]
      # data = data.dropna()
      # print(data.shape)
      # bin_heights, bin_borders = np.histogram(data[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'], bins='auto', normed=True)
      # bin_width = np.diff(bin_borders)
      axs[group - 1].hist(data[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'], bins='auto')
      # axs[group - 1].set_xlabel(r'$\Delta\beta$')
      axs[group - 1].set_ylabel(r'Events')
      axs[group - 1].set_title(r'$\beta$ in ' + f'[{round(x[group - 1] - x_interval_over_2, 3)}, {round(x[group -1 ] + x_interval_over_2, 3)}]')
      axs[group - 1].set_xlim((-0.05, 0.08))

  if to_save:
    filename = f'Hist_{weight}_avg_rw={r_width}_tw={t_width}_rs={r_factor}_ts={t_factor}_rsigms={avg_sigmas[0]}-{avg_sigmas[-1]-1}_tsigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1]-1}_background={background}'
    if deg_lim:
      filename += '_10deg'
    filename += '.png'
    fig.savefig(os.path.join('results', f'{filename}'))
    plt.close(fig)
  else:
    plt.show()

In [83]:
# print([[[]]* 5] * 5)
# print(cal_arr.shape) # r_sigms -> t_sigms -> thetq_groups -> 3 params


In [84]:
def pol2old(x, b, c, d):
  return b * x ** 2 + c * x + d

In [85]:
def pol2(x, p0, p1, p2):
  return p0 + p1 * x + p2 * x ** 2

In [86]:
def d3pol2(X, p0, p1, p2, q0, q1, q2, k0, k1, k2):
  r, theta = X
  # return pol(r, pol2(theta, p0, p1, p2), pol2(theta, q0, q1, q2), pol2(theta, k0, k1, k2))
  return pol(r, p0 + p1 * theta + p2 * theta ** 2, q0 + q1 * theta + q2 * theta ** 2, k0 + k1 * theta + k2 * theta ** 2)

In [87]:
def calibration(edf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, weighed=True, deg_lim=False, param_fit=False):
  def gaussian(x, mean, sigma):
    return norm.pdf(x, loc=mean, scale=sigma)
  # to_return_unbinned = [[], [], [], []]
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  theta_p_max = max(bdf.cos_theta_p)
  theta_p_min = min(bdf.cos_theta_p)
  num_of_theta_intervals = 11
  theta_intervals = np.linspace(theta_p_min, theta_p_max, num=num_of_theta_intervals)
  theta_dif = (theta_intervals[1:] + theta_intervals[:-1]) / 2
  to_return_unbinned = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), num_of_theta_intervals - 1, 3), 0.)

  errs_tmp = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), num_of_theta_intervals - 1, 3), 0.)
  fit_params = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), 3, 3), 0.)

  weight = 'weighed' if weighed else 'unweighed'
  beta_min = min(bdf.beta)
  num_of_beta_intervals = 10 # was 20
  beta_delim = (max(bdf.beta) - min(bdf.beta)) / num_of_beta_intervals
  beta_intervals = np.linspace(min(bdf.beta), max(bdf.beta), num=num_of_beta_intervals + 1)
  dir_to_save = f'{weight}_rw={step}_tw={t_window_width}_rs={r_width_factor}_ts={t_width_factor}'
  if not os.path.exists(os.path.join('calibrations', dir_to_save)):
    os.mkdir(os.path.join('calibrations', dir_to_save))
  for r_sigms in range(*avg_sigmas):
    fig, axs = plt.subplots(num_of_theta_intervals - 1, np.ptp(avg_t_sigmas), figsize=(16 * np.ptp(avg_t_sigmas), 90))
    for t_sigms in range(*avg_t_sigmas):
      chosen_column = f'unfixed_calculated_r_2d_{r_sigms}_rsigms_{t_sigms}_tsigms'

      meas_r_min = min(bdf[chosen_column])
      meas_r_max = max(bdf[chosen_column])
      num_of_meas_r_intervals = num_of_beta_intervals
      meas_r_delim = (meas_r_max - meas_r_min) / num_of_meas_r_intervals
      meas_r_intervals = np.linspace(meas_r_min, meas_r_max, num=num_of_meas_r_intervals + 1)


      # fig2, axs2 = plt.subplots(10* num_of_theta_intervals - 10, 10, figsize=(16, 90))

      for theta_interval_index in range(num_of_theta_intervals - 1):
        xerrs = []
        yerrs = []
        gauss_beta = []
        gauss_r = []
        t_bdf = bdf.copy()
        t_bdf = t_bdf[np.isfinite(t_bdf[chosen_column])]
        t_bdf = t_bdf[t_bdf.signal_counts >= 15]
        t_bdf = t_bdf[t_bdf.cos_theta_p <= theta_intervals[theta_interval_index + 1]]
        t_bdf = t_bdf[t_bdf.cos_theta_p >= theta_intervals[theta_interval_index]]
        if t_sigms - avg_t_sigmas[0] != 0:
          for_colorbar = axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].hist2d(t_bdf[chosen_column], t_bdf.beta, bins=70, range=((0, 80), (beta_min,1)))
          fig.colorbar(for_colorbar[3], ax=axs[theta_interval_index, t_sigms - avg_t_sigmas[0]])
        else:
          for_colorbar = axs[theta_interval_index].hist2d(t_bdf[chosen_column], t_bdf.beta, bins=70, range=((0, 80), (beta_min,1)))
          fig.colorbar(for_colorbar[3], ax=axs[theta_interval_index])
        t_bdf = t_bdf[t_bdf[chosen_column] <= 65]
        t_bdf = t_bdf[t_bdf[chosen_column] >= 25]

        pol_param, cov = curve_fit(pol, t_bdf[chosen_column], t_bdf.beta, maxfev=50000, p0=(1, 80, 30))
        to_return_unbinned[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][theta_interval_index] = pol_param
        pol_param_errs = np.sqrt(np.diag(cov))
        errs_tmp[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][theta_interval_index] = pol_param_errs
        rs = np.linspace(10, 80, num=50)
        chi2 = np.sum((t_bdf.beta - pol(t_bdf[chosen_column], *pol_param)) ** 2)

        if t_sigms - avg_t_sigmas[0] != 0:
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].plot(rs, pol(rs, *pol_param), label=r'$\chi^2$ = '+ str(chi2), c='r')
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_xlim((0, 90))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylim((0.955, 1))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylim((beta_min, 1))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_xlabel(r'$R_{reco}$, mm')
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylabel(r'$\beta_{true}$')
        else:
          axs[theta_interval_index].plot(rs, pol(rs, *pol_param), label=r'$\chi^2$ = '+ str(chi2), c='r')
          axs[theta_interval_index].set_xlim((0, 90))
          axs[theta_interval_index].set_ylim((0.955, 1))
          axs[theta_interval_index].set_ylim((beta_min, 1))
          axs[theta_interval_index].set_xlabel(r'$R_{reco}$, mm')
          axs[theta_interval_index].set_ylabel(r'$\beta_{true}$')
      if param_fit:
        t_bdf = bdf.copy()
        t_bdf = t_bdf[np.isfinite(t_bdf[chosen_column])]
        t_bdf = t_bdf[t_bdf.signal_counts >= 5]
        X = (np.array(t_bdf[chosen_column]), np.array(t_bdf.cos_theta_p))
        fit, errs = curve_fit(d3pol2, X, t_bdf.beta, p0=(1.219, -0.5588, 0.2946, 864.4, -1922, 1055, -2535, 6572, -3751))
        errs = np.sqrt(np.diag(errs))
        for param in range(3):
          fit_params[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][param] = fit[param * 3: param * 3 + 3]
        print(fit)
        print(errs)
        chi2 = np.sum((t_bdf.beta - d3pol2(X, *fit)) ** 2)
        print(chi2)
      # if param_fit:
      #   for param in range(3):
      #     fit, _ = curve_fit(pol2, theta_dif, to_return_unbinned[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][:, param], sigma=errs_tmp[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][:, param])
      #     fit_params[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][param] = fit
    filename = f'rsigm={r_sigms}_t_sigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1] - 1}'
    if deg_lim:
      filename += '_10deg'
    if dir_to_save != '':
      fig.savefig(os.path.join('calibrations', dir_to_save, f'{filename}'))
      plt.close(fig)


  if param_fit:
    return fit_params
  return to_return_unbinned
# cal_arr = np.array(calibration(edf, bdf, avg_sigmas=(1, 5), avg_t_sigmas=(1, 5))) # add r and t calibr - done

In [88]:
ы

NameError: name 'ы' is not defined

In [ ]:
for i in range(2):
  print(i)

# With DCR

In [89]:
event = 1444
#momentum = edf.at[(event, 0), 'momentum']
#theta_p = edf.at[(event, 0), 'theta_p']*180/np.pi
avg_sigmas=(4, 5)
avg_t_sigmas=(4, 5)
num_of_groups = 10
r_width = float(idf.pixel_size)
t_width = 0.25
t_step = 0.25
r_factor = 2 # not to change
t_factor = int(t_width / t_step)
weighed = False
deg_lim = False
param_fit = False
for t_width in [0.25]: # [0.25, 0.5, 0.75, 1]
  for t_step in [0.25]: # [0.25, 0.5]
    for r_width in [2 * float(idf.pixel_size)]: # [float(idf.pixel_size), 2 * float(idf.pixel_size)]
      for weighed in [True]: # [False, True]
        print(f'Generating: {t_width}_{t_step}_{r_width}_{weighed}\n')
        cal_arr_for_dcr = False
        beta_sigms = []
        beta_sigms_yerr = []
        beta_sigms_deglim = []
        beta_sigms_yerr_deglim = []
        for dcr in ['0']  : # ['0', '1e3', '1e4', '1e5', '1e6']
          timer_start = perf_counter()
          gen = genChunkFromRoot(filepath, 10000, noisefreqpersqmm=float(dcr), shiftSignalTimes=True)
          c_bdf_d = pd.DataFrame()
          for i in range(1):
            edf_d = next(gen)
            # edf_d = next(genChunkFromRoot(filepath, 10000, noisefreqpersqmm=float(dcr), shiftSignalTimes=True))

            bdf_d = pd.DataFrame()
            # signalLength(edf_d)
            recoAngles(edf_d, idf)
            edf_d = applySpaceCut(edf_d)
            planeRecalculation(edf_d, idf)
            planeRotation(edf_d)
            edf_d = applySecondSpaceCut(edf_d)
            primaryDirectionRecalculation(edf_d)

            recoAngles(edf_d, idf, rotation_mode=True)
            cal_arr_for_dcr = rSlidingWindow(edf_d, idf, bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, cal_arr=cal_arr_for_dcr, num_of_groups=num_of_groups,
                                             step=r_width, t_window_width=t_width, r_width_factor=r_factor, t_width_factor=t_factor, weighed=weighed, deg_lim=deg_lim, param_fit=param_fit)

            if deg_lim:
              edf_d = edf_d[edf_d.theta_p <= 10. * np.pi / 180]
              edf_d = edf_d[edf_d.signal_counts >= 5]
              bdf_d = bdf_d[bdf_d.theta_p <= 10. * np.pi / 180]
              bdf_d = bdf_d[bdf_d.signal_counts >= 5]
            if i == 0:
              c_bdf_d = bdf_d
            else:
              c_bdf_d = pd.concat([c_bdf_d, bdf_d], ignore_index=True)
          plot_groupped_distributions(c_bdf_d, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background=dcr, deg_lim=False, num_of_groups=num_of_groups, to_save=True)
          bg = betaGroupsRMS90(c_bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, n=num_of_groups)
          beta_sigms.append(bg[0])
          beta_sigms_yerr.append(bg[2])

          c_bdf_d = c_bdf_d[c_bdf_d.theta_p <= 10. * np.pi / 180]
          c_bdf_d = c_bdf_d[c_bdf_d.signal_counts >= 5]

          plot_groupped_distributions(c_bdf_d, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background=dcr, deg_lim=True, num_of_groups=num_of_groups, to_save=True)
          bg_deglim = betaGroupsRMS90(c_bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, n=num_of_groups)
          beta_sigms_deglim.append(bg_deglim[0])
          beta_sigms_yerr_deglim.append(bg_deglim[2])
          print(perf_counter() - timer_start)

          # plothits(edf_d, 1, event, dir_to_save=f'event_{event}_{dcr}_noise')

          #figxy.savefig(os.path.join(picsdir, f'labeled_ring_pi_p{momentum:.0f}mev_theta{theta_p:.0f}deg_dcr{dcr:.3g}.png'))
          #figtime.savefig(os.path.join(picsdir, f'labeled_time_pi_p{momentum:.0f}mev_theta{theta_p:.0f}deg_dcr{dcr:.3g}.png'))
        beta_sigms = np.array(beta_sigms)
        beta_sigms_yerr = np.array(beta_sigms_yerr)
        beta_sigms_deglim = np.array(beta_sigms_deglim)
        beta_sigms_yerr_deglim = np.array(beta_sigms_yerr_deglim)
        plot_final_graph(beta_sigms, beta_sigms_yerr, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, deg_lim=False, num_of_groups=num_of_groups)

        plot_final_graph(beta_sigms_deglim, beta_sigms_yerr_deglim, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, deg_lim=True, num_of_groups=num_of_groups)

Generating: 0.25_0.25_6.32_True

    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
29.183363300000565


In [ ]:
cal_arr_for_dcr